In [ ]:
from 

In [ ]:
from fastai2.basics import *
from fastai2.text.all import *

In [ ]:
source = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
items = [1, 'hello', 1.2, 5]

In [ ]:
class TypeDispatch2(TypeDispatch):
    def __call__(self, *args, **kwargs):
        ts = L(args).map(type)[:2]
        f = self[tuple(ts)]
        if not f: return None
        if self.inst is not None: f = MethodType(f, self.inst)
        return f(*args, **kwargs)

In [ ]:
class DispatchReg2(DispatchReg):
    def __init__(self): self.d = defaultdict(TypeDispatch2)

In [ ]:
typedispatch2 = DispatchReg2()

In [ ]:
type(fint)

__main__.TypeDispatch2

In [ ]:
@typedispatch2
def fint(x:int): return x+1
@typedispatch2
def fstr(x:str): return 'hello ' + str(x)
@typedispatch2
def fflt(x:float): return x/2
@typedispatch2
def flst(x:list): return x

In [ ]:
@typedispatch
def flt2int(x:float): return int(x)
@typedispatch
def int2str(x:int): return str(x)
@typedispatch
def int2flt(x:int): return float(x)
@typedispatch
def flt2str(x:float): return str(x)
@typedispatch
def str2int(x:str): return int(float(x))

In [ ]:
TypeDispatch??

In [ ]:
lf = lfs[0]
[v.__name__ for k in lf.funcs.d for l,v in lf.funcs[k].d.items()]

['fint']

In [ ]:
{
    0: [..,..,..]
    1: [..,..,..]
}

In [ ]:
def reduce_lbls(lbls):
    lbls = lbls.filter(partial(equals, None), negate=True) # Remove all Nones
    if len(lbls)>1: raise ValueError # TODO: Should figure out this case
    return lbls[0] if len(lbls)==1 else 'ABSTAIN'

In [ ]:
def compose_tfms2(x, tfms, is_enc=True, reverse=False, **kwargs):
    "Apply all `func_nm` attribute of `tfms` on `x`, maybe in `reverse` order"
    if reverse: tfms = reversed(tfms)
    lbls = defaultdict(L)
    for i,lf in enumerate(lfs): lbls[i].append(lf(x))
    for f in tfms:
        if not is_enc: f = f.decode
        x = f(x, **kwargs)
        for i,lf in enumerate(lfs): lbls[i].append(lf(x))
    # TODO: LF can be called twice (if types repeat)
    lbls = {k:reduce_lbls(v) for k,v in lbls.items()} # This is how you reduce it
    return lbls

In [ ]:
class Pipeline2(Pipeline):
    def __call__(self, o): return compose_tfms2(o, tfms=self.fs, split_idx=self.split_idx)

In [ ]:
lfs = [fint, fstr, fflt, flst]

In [ ]:
tfms = Pipeline2([flt2str, str2int])

In [ ]:
tfms(1.2)

{0: 2, 1: 'hello 1.2', 2: 0.6, 3: 'ABSTAIN'}

In [ ]:
fflt(1, 3)

1

In [ ]:
class Pipeline:
    "A pipeline of composed (for encode/decode) transforms, setup with types"
    def __init__(self, funcs=None, split_idx=None):
        self.split_idx,self.default = split_idx,None
        if isinstance(funcs, Pipeline): self.fs = funcs.fs
        else:
            if isinstance(funcs, Transform): funcs = [funcs]
            self.fs = L(ifnone(funcs,[noop])).map(mk_transform).sorted(key='order')
        for f in self.fs:
            name = camel2snake(type(f).__name__)
            a = getattr(self,name,None)
            if a is not None: f = L(a)+f
            setattr(self, name, f)

    def setup(self, items=None, train_setup=False):
        tfms = self.fs[:]
        self.fs.clear()
        for t in tfms: self.add(t,items, train_setup)

    def add(self,t, items=None, train_setup=False):
        t.setup(items, train_setup)
        self.fs.append(t)

    def __call__(self, o): return compose_tfms2(o, tfms=self.fs, split_idx=self.split_idx)
    def __repr__(self): return f"Pipeline: {' -> '.join([f.name for f in self.fs if f.name != 'noop'])}"
    def __getitem__(self,i): return self.fs[i]
    def __setstate__(self,data): self.__dict__.update(data)
    def __getattr__(self,k): return gather_attrs(self, k, 'fs')
    def __dir__(self): return super().__dir__() + gather_attr_names(self, 'fs')

    def decode  (self, o, full=True):
        if full: return compose_tfms(o, tfms=self.fs, is_enc=False, reverse=True, split_idx=self.split_idx)
        #Not full means we decode up to the point the item knows how to show itself.
        for f in reversed(self.fs):
            if self._is_showable(o): return o
            o = f.decode(o, split_idx=self.split_idx)
        return o

    def show(self, o, ctx=None, **kwargs):
        o = self.decode(o, full=False)
        o1 = (o,) if not _is_tuple(o) else o
        if hasattr(o, 'show'): ctx = o.show(ctx=ctx, **kwargs)
        else:
            for o_ in o1:
                if hasattr(o_, 'show'): ctx = o_.show(ctx=ctx, **kwargs)
        return ctx

    def _is_showable(self, o):
        if hasattr(o, 'show'): return True
        if _is_tuple(o): return all(hasattr(o_, 'show') for o_ in o)
        return False